# Wellpathpy tutorial

This document aims to provide a simple workthrough using `wellpathpy` showing:

- the abbreviations used in `wellpathpy`
- required imports
- loading a deviation
- loading a well header
- converting units
- choosing an md to tvd conversion method
- choosing a depth interval to resample to a given step
- exporting the results to \*.csv

## Abbreviations

- m: metres
- ft: feet
- md: measured depth
- inc: inclination
- azi: azimuth
- tvd: true vertical depth
- east_offset: horizontal distance away from wellhead towards the east
- north_offset: horizontal distance away from wellhead towards the north
- tvdss: true vertical depth subsea
- mE: horizontal distance in meters away from surface location towards the east
- mN: horizontal distance in meters away from surface location towards the north


## Imports

`wellpathpy` depends on [numpy](numpy.org), [pandas](pandas.pydata.org) and [pint](github.com/hgrecco/pint).

```
import numpy as np
import pandas as pd
import wellpathpy as wp
from wellpathpy import interpolate_deviation, interpolate_position # rename these ?
```

## Loading a deviation

Assuming the file is a csv in md/inc/azi, you can load it into python with:

```
md, inc, azi = wp.read_csv(fname)
```
We depend on Pandas and so we support all arguments `pd.read_csv()` in pandas does.

## Loading the well header

Assuming the well header information in a \*.json file with this format:
```
{
"datum": "kb",
"elevation_units": "m",
"elevation": 100.0,
"surface_coordinates_units": "m",
"surface_easting": 1000.0,
"surface_northing": 2000.0
}
```

we can load it with:

```
header = wp.read_header_json(fname)
```

## Converting units

By default, no unit conversions are run by `wellpathpy`, you therefore have the following options:

1. Import your deviations and headers in consistent SI units where:
    - `md`: meters
    - `inc`: degrees
    - `azi`: degrees
    - `elevation`: meters above mean sea level
    - `surface_easting`: meters east of reference point
    - `surface_northing`: meters north of reference point


2. Import deviations and headers in other units (e.g. feet) and convert to SI units:
    - `md`: feet
    - `inc`: degrees
    - `azi`: degrees
    - `elevation`: feet above mean sea level
    - `surface_easting`: feet east of reference point
    - `surface_northing`: feet north of reference point
    
**Important Notes:**  
- The units for [`elevation`, `surface_northing`, `surface_easting`] must be the same as the `md` units before any `md->tvd` calculations are run, otherwise you will get inconsistent results.
- `inc` and `azi` must always be passed as `degrees`, otherwise erroneous results will be returned.

### Conversion API

In order to convert `md`, `elevation`, `surface_easting` or `surface_northing` from 'ft' to 'm' where `elevation_units` and `surface_coordinates_units` are in 'ft' for example, run:

```
md                         = wp.unit_convert(md, src='ft', dst='m')

elevation_units            = header['elevation_units']
surface_coordinates_units  = header['surface_coordinates_units']

header['elevation']        = wp.unit_convert(header['elevation'],
                                             src=elevation_units,
                                             dst='m')
header['surface_easting']  = wp.unit_convert(header['surface_easting'],
                                             src=surface_coordinates_units,
                                             dst='m')
header['surface_northing'] = wp.unit_convert(header['surface_northing'],
                                             src=surface_coordinates_units, 
                                             dst='m')
```

We depend on [pint](github.com/hgrecco/pint) for the unit conversions

In [ ]:
# create a header dictionnary
# I'm adding surface norhting and easting
header_dict = wp.get_header(datum='kb', units='m', elevation=0., surface_northing=0., surface_easting=0.)

In [ ]:
# load a well deviation in md, inc, azi
deviation = wp.read_csv(md, inc, azi, *args)

In [ ]:
# once the deviation is loaded, choose method to ouput deviation and positional DataFrame
# the deviation is returned as an offset log, this is what we currently return without the location() function
# the positional log is returned thanks to the header attribute and location() function
# if no header information is passed in `wb.get_header()`, default values are used
wb.mincurve(header_dict)
>>> pd.DataFrame({'md': np.array(md),
               'inc': np.array(inc),
               'azi': np.array(azi),
               'tvd': np.array(tvd),
               'east_offset': np.array(east_offset),
               'north_offset': np.array(north_offset),
               'tvdss': np.array(tvdss),
               'mE': np.array(mE),
               'mN': np.array(mN),
              })

wb.radcurve(header_dict)
>>> pd.DataFrame({'md': np.array(md),
               'inc': np.array(inc),
               'azi': np.array(azi),
               'tvd': np.array(tvd),
               'east_offset': np.array(east_offset),
               'north_offset': np.array(north_offset),
               'tvdss': np.array(tvdss),
               'mE': np.array(mE),
               'mN': np.array(mN),
              })

wb.baltan(header_dict)
>>> pd.DataFrame({'md': np.array(md),
             'inc': np.array(inc),
             'azi': np.array(azi),
             'tvd': np.array(tvd),
             'east_offset': np.array(east_offset),
             'north_offset': np.array(north_offset),
             'tvdss': np.array(tvdss),
             'mE': np.array(mE),
             'mN': np.array(mN),
             })

wb.hightan(header_dict)
>>> pd.DataFrame({'md': np.array(md),
             'inc': np.array(inc),
             'azi': np.array(azi),
             'tvd': np.array(tvd),
             'east_offset': np.array(east_offset),
             'north_offset': np.array(north_offset),
             'tvdss': np.array(tvdss),
             'mE': np.array(mE),
             'mN': np.array(mN),
             })

wb.lowtan(header_dict)
>>> pd.DataFrame({'md': np.array(md),
             'inc': np.array(inc),
             'azi': np.array(azi),
             'tvd': np.array(tvd),
             'east_offset': np.array(east_offset),
             'north_offset': np.array(north_offset),
             'tvdss': np.array(tvdss),
             'mE': np.array(mE),
             'mN': np.array(mN),
             })

wb.avgtan(header_dict)
>>> pd.DataFrame({'md': np.array(md),
             'inc': np.array(inc),
             'azi': np.array(azi),
             'tvd': np.array(tvd),
             'east_offset': np.array(east_offset),
             'north_offset': np.array(north_offset),
             'tvdss': np.array(tvdss),
             'mE': np.array(mE),
             'mN': np.array(mN),
             })

In [ ]:
# resample well deviation in md to given step
wb.resample_md(md_step=1)
>>> interpolate_deviation(md, inc, azi, md_step=1)
>>> pd.DataFrame({'md': np.array(md), 'inc': np.array(inc), 'azi': np.array(azi)})

In [ ]:
# resample section of well positional log in tvd to given step
section_max = int
wb.resample_tvd(tvd_start=0., tvd_stop=np.array(tvd[section_max]), tvd_step=1)
# previous line seems wrong. I want to pass a slice of the positional log to wb.resample_tvd
>>> interpolate_position(tvd, easting, northing, tvd_step=1)
>>> pd.DataFrame({'tvd': np.array(tvd), 'easting': np.array(easting), 'northing': np.array(northing)})